In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
# the number of segments; when we deploy repeater
N = np.array([1,2,4,8,16,32,64,128])

# metal parameter
metal_r = 120 / 1e-6
metal_c = (0.2) * 1e-15 / 1e-6
WL_length = 256 * 1e-6

# cell parameter
cell_num = 256
cell_c = 1e-15
cell_C = cell_c * cell_num

###################################################################
### TR/Logic Gates parameters: based on P/N ratio 7:3 (2.57:1.1)###
###################################################################

# TR parameter
# gate cap
tau = 1905/448 * 1e-12 #measured FO1 delay : tau(1+pinv)
TR_C = (1.5/1.1) * 1e-15 # measured
#tau = 3 * TR_C * TR_R
TR_R = tau / (3*TR_C)




# Logic Gates
g_inv, p_inv = 1, 321/127

g_nand2, g_nand3, g_nand6 = 4/3 , 5/3, 8/3
p_nand2, p_nand3, p_nand6 = p_inv*2, p_inv*3, p_inv*6

g_nor2, g_nor3 = 5/3, 7/3
p_nor2, p_nor3 = p_inv*2, p_inv*3

# Decoder Logic
G_nand6_inv = g_nand6 * g_inv
P_nand6_inv = p_nand6 + p_inv
G_nand3_nor2 = g_nand3 * g_nor2
P_nand3_nor2 = p_nand3 + p_nor2
B = 32 # critical path: CLK signal

In [3]:
# delay of decoder
WL_R = metal_r * WL_length
WL_C = metal_c * WL_length

# NMOS width of repeater derived from RC delay of N segments
W = ( N*(TR_R*(metal_c + 1e-15))/(3*metal_r*TR_C) ) ** (1/2)

C_out = (WL_C + cell_C)/N + 3 * W * TR_C
C_in = 5 * 1e-15 * 32

H = C_out/C_in

G = np.array([G_nand6_inv, G_nand3_nor2])
F = np.outer(G,H)*B

F = pd.DataFrame(data = F, columns = [1,2,4,8,16,32,64,128], index=['NAND6-INV','NAND3-NOR2'])
stages = np.log(F)/np.log(4)
stages

,1,2,4,8,16,32,64,128
NAND6-INV,3.684296,3.195540,2.726423,2.307233,1.996351,1.858589,1.899468,2.058158
NAND3-NOR2,3.713743,3.224986,2.755870,2.336680,2.025798,1.888036,1.928915,2.087605


In [4]:
#with 2 stages
f_2 = F ** (1/2)
delay_decoder_stage2 = 2*f_2
delay_decoder_stage2.loc['NAND6-INV'] = delay_decoder_stage2.loc['NAND6-INV'] + P_nand6_inv
delay_decoder_stage2.loc['NAND3-NOR2'] = delay_decoder_stage2.loc['NAND3-NOR2'] + P_nand3_nor2
delay_decoder_stage2 = delay_decoder_stage2 * tau
new_row_names = {'NAND6-INV':'NAND6-INV Stage2','NAND3-NOR2':'NAND3-NOR2 Stage2'}
delay_decoder_stage2.rename(index = new_row_names, inplace = True)
delay_decoder_stage2

,1,2,4,8,16,32,64,128
NAND6-INV Stage2,1.845622e-10,1.531456e-10,1.315180e-10,1.173258e-10,1.091663e-10,1.060761e-10,1.069625e-10,1.106516e-10
NAND3-NOR2 Stage2,1.653211e-10,1.332567e-10,1.111831e-10,9.669818e-11,8.837047e-11,8.521649e-11,8.612118e-11,8.988636e-11


In [5]:
#with 4 stages
f_4 = F ** (1/4)
delay_decoder_stage4 = 4*f_4
delay_decoder_stage4.loc['NAND6-INV'] = delay_decoder_stage4.loc['NAND6-INV'] + P_nand6_inv + 2 * p_inv
delay_decoder_stage4.loc['NAND3-NOR2'] = delay_decoder_stage4.loc['NAND3-NOR2'] + P_nand3_nor2 + 2 * p_inv
delay_decoder_stage4 = delay_decoder_stage4 * tau
new_row_names = {'NAND6-INV':'NAND6-INV Stage4','NAND3-NOR2':'NAND3-NOR2 Stage4'}
delay_decoder_stage4.rename(index = new_row_names, inplace = True)
delay_decoder_stage4

,1,2,4,8,16,32,64,128
NAND6-INV Stage4,1.577143e-10,1.482117e-10,1.404866e-10,1.345698e-10,1.307048e-10,1.291208e-10,1.295829e-10,1.314404e-10
NAND3-NOR2 Stage4,1.368443e-10,1.272442e-10,1.194399e-10,1.134624e-10,1.095577e-10,1.079575e-10,1.084244e-10,1.103009e-10


In [6]:
#with 6 stages
f_6 = F ** (1/6)
delay_decoder_stage6 = 6*f_6
delay_decoder_stage6.loc['NAND6-INV'] = delay_decoder_stage6.loc['NAND3-NOR2'] + P_nand6_inv + 4 * p_inv
delay_decoder_stage6.loc['NAND3-NOR2'] = delay_decoder_stage6.loc['NAND3-NOR2'] + P_nand3_nor2 + 4 * p_inv
delay_decoder_stage6 = delay_decoder_stage6 * tau
new_row_names = {'NAND6-INV':'NAND6-INV Stage6','NAND3-NOR2':'NAND3-NOR2 Stage6'}
delay_decoder_stage6.rename(index = new_row_names, inplace = True)
delay_decoder_stage6

,1,2,4,8,16,32,64,128
NAND6-INV Stage6,1.784006e-10,1.719749e-10,1.664537e-10,1.620017e-10,1.589676e-10,1.576912e-10,1.580657e-10,1.595535e-10
NAND3-NOR2 Stage6,1.569051e-10,1.504794e-10,1.449582e-10,1.405061e-10,1.374720e-10,1.361956e-10,1.365702e-10,1.380580e-10


In [7]:
#with 8 stages
f_8 = F ** (1/8)
delay_decoder_stage8 = 8*f_8
delay_decoder_stage8.loc['NAND6-INV'] = delay_decoder_stage8.loc['NAND6-INV'] + P_nand6_inv + 6 * p_inv
delay_decoder_stage8.loc['NAND3-NOR2'] = delay_decoder_stage8.loc['NAND3-NOR2'] + P_nand3_nor2 + 6 * p_inv
delay_decoder_stage8 = delay_decoder_stage8 * tau
new_row_names = {'NAND6-INV':'NAND6-INV Stage8','NAND3-NOR2':'NAND3-NOR2 Stage8'}
delay_decoder_stage8.rename(index = new_row_names, inplace = True)
delay_decoder_stage8

,1,2,4,8,16,32,64,128
NAND6-INV Stage8,2.041346e-10,1.989037e-10,1.942830e-10,1.904602e-10,1.877991e-10,1.866649e-10,1.869987e-10,1.883168e-10
NAND3-NOR2 Stage8,1.829686e-10,1.777110e-10,1.730666e-10,1.692242e-10,1.665495e-10,1.654096e-10,1.657450e-10,1.670699e-10


In [8]:
delay_decoder = pd.concat([delay_decoder_stage2, delay_decoder_stage4, delay_decoder_stage6, delay_decoder_stage8])
delay_decoder

,1,2,4,8,16,32,64,128
NAND6-INV Stage2,1.845622e-10,1.531456e-10,1.315180e-10,1.173258e-10,1.091663e-10,1.060761e-10,1.069625e-10,1.106516e-10
NAND3-NOR2 Stage2,1.653211e-10,1.332567e-10,1.111831e-10,9.669818e-11,8.837047e-11,8.521649e-11,8.612118e-11,8.988636e-11
NAND6-INV Stage4,1.577143e-10,1.482117e-10,1.404866e-10,1.345698e-10,1.307048e-10,1.291208e-10,1.295829e-10,1.314404e-10
NAND3-NOR2 Stage4,1.368443e-10,1.272442e-10,1.194399e-10,1.134624e-10,1.095577e-10,1.079575e-10,1.084244e-10,1.103009e-10
NAND6-INV Stage6,1.784006e-10,1.719749e-10,1.664537e-10,1.620017e-10,1.589676e-10,1.576912e-10,1.580657e-10,1.595535e-10
NAND3-NOR2 Stage6,1.569051e-10,1.504794e-10,1.449582e-10,1.405061e-10,1.374720e-10,1.361956e-10,1.365702e-10,1.380580e-10
NAND6-INV Stage8,2.041346e-10,1.989037e-10,1.942830e-10,1.904602e-10,1.877991e-10,1.866649e-10,1.869987e-10,1.883168e-10
NAND3-NOR2 Stage8,1.829686e-10,1.777110e-10,1.730666e-10,1.692242e-10,1.665495e-10,1.654096e-10,1.657450e-10,1.670699e-10


In [9]:
segment_delay = (TR_R/W) * ( (3*TR_C*p_inv*W) + cell_C/(2*N) + WL_C/(2*N) ) + (TR_R/W + WL_R/N) * ( (3*TR_C*W) + cell_C/(2*N) + WL_C/(2*N) )
last_segment_delay = (TR_R/W) * ( (3*TR_C*p_inv*W) + cell_C/(2*N) + WL_C/(2*N) ) + (TR_R/W + WL_R/N) * (cell_C/(2*N) + WL_C/(2*N))
repeater_delay = 15 *1e-12 #measured
delay_WL = (N-1) * segment_delay + last_segment_delay   + (N-1) * repeater_delay
delay_WL

array([5.22002641e-09, 2.80483988e-09, 1.64841136e-09, 1.18645462e-09,
       1.18501211e-09, 1.62311478e-09, 2.67984103e-09, 4.81900644e-09])

In [10]:
delay_total = delay_decoder.copy()
delay_total = delay_total + delay_WL
delay_total

,1,2,4,8,16,32,64,128
NAND6-INV Stage2,5.404589e-09,2.957986e-09,1.779929e-09,1.303780e-09,1.294178e-09,1.729191e-09,2.786803e-09,4.929658e-09
NAND3-NOR2 Stage2,5.385347e-09,2.938097e-09,1.759594e-09,1.283153e-09,1.273383e-09,1.708331e-09,2.765962e-09,4.908893e-09
NAND6-INV Stage4,5.377741e-09,2.953052e-09,1.788898e-09,1.321024e-09,1.315717e-09,1.752236e-09,2.809424e-09,4.950447e-09
NAND3-NOR2 Stage4,5.356871e-09,2.932084e-09,1.767851e-09,1.299917e-09,1.294570e-09,1.731072e-09,2.788265e-09,4.929307e-09
NAND6-INV Stage6,5.398427e-09,2.976815e-09,1.814865e-09,1.348456e-09,1.343980e-09,1.780806e-09,2.837907e-09,4.978560e-09
NAND3-NOR2 Stage6,5.376932e-09,2.955319e-09,1.793370e-09,1.326961e-09,1.322484e-09,1.759310e-09,2.816411e-09,4.957064e-09
NAND6-INV Stage8,5.424161e-09,3.003744e-09,1.842694e-09,1.376915e-09,1.372811e-09,1.809780e-09,2.866840e-09,5.007323e-09
NAND3-NOR2 Stage8,5.402995e-09,2.982551e-09,1.821478e-09,1.355679e-09,1.351562e-09,1.788524e-09,2.845586e-09,4.986076e-09


In [11]:
delay_total.describe()

,1,2,4,8,16,32,64,128
count,8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00
mean,5.390883e-09,2.962456e-09,1.796085e-09,1.326986e-09,1.321086e-09,1.757406e-09,2.814650e-09,4.955916e-09
std,2.088651e-11,2.387420e-11,2.833137e-11,3.163464e-11,3.351311e-11,3.419934e-11,3.400455e-11,3.317704e-11
min,5.356871e-09,2.932084e-09,1.759594e-09,1.283153e-09,1.273383e-09,1.708331e-09,2.765962e-09,4.908893e-09
25%,5.377538e-09,2.949313e-09,1.776910e-09,1.302815e-09,1.294472e-09,1.730602e-09,2.787900e-09,4.929570e-09
50%,5.391887e-09,2.956652e-09,1.791134e-09,1.323993e-09,1.319101e-09,1.755773e-09,2.812918e-09,4.953756e-09
75%,5.403393e-09,2.978249e-09,1.816518e-09,1.350262e-09,1.345875e-09,1.782736e-09,2.839827e-09,4.980439e-09
max,5.424161e-09,3.003744e-09,1.842694e-09,1.376915e-09,1.372811e-09,1.809780e-09,2.866840e-09,5.007323e-09


In [12]:
H

array([1.93663849, 0.98353038, 0.51327698, 0.28706076, 0.18655397,
       0.15412152, 0.16310794, 0.20324305])

In [13]:
f_2

,1,2,4,8,16,32,64,128
NAND6-INV,12.855342,9.161219,6.618129,4.949328,3.989896,3.626528,3.730756,4.164542
NAND3-NOR2,13.120428,9.350130,6.754600,5.051387,4.072171,3.701309,3.807687,4.250417


In [14]:
W

array([0.65074991, 0.92029934, 1.30149981, 1.84059869, 2.60299963,
       3.68119738, 5.20599926, 7.36239476])

In [15]:
2*2.60299963

5.20599926

In [16]:
C_out

array([3.09862159e-13, 1.57364861e-13, 8.21243174e-14, 4.59297219e-14,
       2.98486348e-14, 2.46594438e-14, 2.60972697e-14, 3.25188876e-14])